# 1. TRENDING STOCK SYMBOLS

API domain: https://yfapi.net
API path: /v1/finance/trending/{region}

Pull data (extract) from yahoofinancials API (URL path is /v1/finance/trending/{region}).
It returns 20 stock symbols based on the region in the URL.

Uses your basic subscription API key obtained from yahoofinancials web site.
Current region: US

In [1]:
# Import required modules
import pandas as pd

In [2]:
##  Ensure you have your own config.py file in same folder so you can import your api key
##   and PostgreSQL database connection string variables
import requests
import pprint
from config import API_KEY
from config import username, password, host, port, database

region_selected = "US"
url = "https://yfapi.net/v1/finance/trending/" + region_selected
## Alternate url
# url = "https://rest.yahoofinanceapi.com/v1/finance/trending/" + region_selected

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [3]:
## Make API request (no query string required for this API)
response1 = requests.request("GET", url, headers=headers).json()

# What does the response look like?  We will have to use indexing to get to the 'result' level
pprint.pprint(response1)

{'finance': {'error': None,
             'result': [{'count': 20,
                         'jobTimestamp': 1632280207071,
                         'quotes': [{'symbol': 'ADBE'},
                                    {'symbol': 'QS'},
                                    {'symbol': 'DIS'},
                                    {'symbol': 'MRIN'},
                                    {'symbol': 'FDX'},
                                    {'symbol': 'UBER'},
                                    {'symbol': 'RWLK'},
                                    {'symbol': 'GTEK'},
                                    {'symbol': 'DKNG'},
                                    {'symbol': '000001.SS'},
                                    {'symbol': 'NNVC'},
                                    {'symbol': 'SFIX'},
                                    {'symbol': 'HLBZ'},
                                    {'symbol': 'UPST'},
                                    {'symbol': 'LCID'},
                                    {

# Optional: Check that 'error' was 'None'
Insert code to show error code, does an assert that code ran fine, shows number of tickers received

In [4]:
## How many responses did we get?  Single result with multiple columns
len(response1['finance']['result'])

1

In [5]:
## Parse through the resulting dictionary tree to get at the 'results'.  Look at only the first
##  result in order to see what columns we received.
response1['finance']['result'][0]

{'count': 20,
 'quotes': [{'symbol': 'ADBE'},
  {'symbol': 'QS'},
  {'symbol': 'DIS'},
  {'symbol': 'MRIN'},
  {'symbol': 'FDX'},
  {'symbol': 'UBER'},
  {'symbol': 'RWLK'},
  {'symbol': 'GTEK'},
  {'symbol': 'DKNG'},
  {'symbol': '000001.SS'},
  {'symbol': 'NNVC'},
  {'symbol': 'SFIX'},
  {'symbol': 'HLBZ'},
  {'symbol': 'UPST'},
  {'symbol': 'LCID'},
  {'symbol': 'XELA'},
  {'symbol': 'ABNB'},
  {'symbol': 'UPS'},
  {'symbol': 'AEMD'},
  {'symbol': 'NURO'}],
 'jobTimestamp': 1632280207071,
 'startInterval': 202109220200}

In [6]:
# Create a new DataFrame from all the results in the dictionary
response1_df = pd.DataFrame(response1['finance']['result'][0]['quotes'])
response1_df.head()

,symbol
0,ADBE
1,QS
2,DIS
3,MRIN
4,FDX


In [10]:
response1_df['startInterval'] = response1['finance']['result'][0]['startInterval']
response1_df['jobTimestamp'] = response1['finance']['result'][0]['jobTimestamp']
response1_df

,symbol,startInterval,jobTimestamp
0,ADBE,202109220200,1632280207071
1,QS,202109220200,1632280207071
2,DIS,202109220200,1632280207071
3,MRIN,202109220200,1632280207071
4,FDX,202109220200,1632280207071
5,UBER,202109220200,1632280207071
6,RWLK,202109220200,1632280207071
7,GTEK,202109220200,1632280207071
8,DKNG,202109220200,1632280207071
9,000001.SS,202109220200,1632280207071


In [8]:
# Extraction of the timestamp, startInterval
from datetime import datetime
start_interval = response1_df['startInterval'].to_string()
job_timestamp = response1_df['jobTimestamp']

job_timestamp = pd.to_datetime(job_timestamp, unit='ms')
# start_interval = datetime.strptime(start_interval, '%Y%m%d%H%M')
start_interval, job_timestamp

('0     202109220200\n1     202109220200\n2     202109220200\n3     202109220200\n4     202109220200\n5     202109220200\n6     202109220200\n7     202109220200\n8     202109220200\n9     202109220200\n10    202109220200\n11    202109220200\n12    202109220200\n13    202109220200\n14    202109220200\n15    202109220200\n16    202109220200\n17    202109220200\n18    202109220200\n19    202109220200',
 0    2021-09-22 03:10:07.071
 1    2021-09-22 03:10:07.071
 2    2021-09-22 03:10:07.071
 3    2021-09-22 03:10:07.071
 4    2021-09-22 03:10:07.071
 5    2021-09-22 03:10:07.071
 6    2021-09-22 03:10:07.071
 7    2021-09-22 03:10:07.071
 8    2021-09-22 03:10:07.071
 9    2021-09-22 03:10:07.071
 10   2021-09-22 03:10:07.071
 11   2021-09-22 03:10:07.071
 12   2021-09-22 03:10:07.071
 13   2021-09-22 03:10:07.071
 14   2021-09-22 03:10:07.071
 15   2021-09-22 03:10:07.071
 16   2021-09-22 03:10:07.071
 17   2021-09-22 03:10:07.071
 18   2021-09-22 03:10:07.071
 19   2021-09-22 03:10:07.0

In [21]:
###(Fix this part of the code in later version)
# Loop through each stock symbol and make a list
## Using 'for' loop
# stock_list=[]
# for stock in response1_df['symbol']:
#     #print(stock)
#     stock_list.append(stock)
## Using list comprehension notation
# stock_list = [stock for stock in response1_df['symbol']]
stock_list = [stock for stock in response1_df['symbol']]
stock_list

['ADBE',
 'QS',
 'DIS',
 'MRIN',
 'FDX',
 'UBER',
 'RWLK',
 'GTEK',
 'DKNG',
 '000001.SS',
 'NNVC',
 'SFIX',
 'HLBZ',
 'UPST',
 'LCID',
 'XELA',
 'ABNB',
 'UPS',
 'AEMD',
 'NURO']

In [22]:
# The quote API query wants the list of symbols as a single comma separated string 
#  with no spaces.  Max is 10 per request.
query_string=''
for stock in stock_list:
    query_string += f"{stock},"
query_string = query_string.rstrip(',')
query_string

'ADBE,QS,DIS,MRIN,FDX,UBER,RWLK,GTEK,DKNG,000001.SS,NNVC,SFIX,HLBZ,UPST,LCID,XELA,ABNB,UPS,AEMD,NURO'

In [25]:
response1_df['symbol']

0          ADBE
1            QS
2           DIS
3          MRIN
4           FDX
5          UBER
6          RWLK
7          GTEK
8          DKNG
9     000001.SS
10         NNVC
11         SFIX
12         HLBZ
13         UPST
14         LCID
15         XELA
16         ABNB
17          UPS
18         AEMD
19         NURO
Name: symbol, dtype: object

In [27]:
###(Fix this part of the code in later version)
# For now, get the first 10 and run the quote API
query_string=''
for i in range(10):
    query_string += f"{response1_df['symbol'][i]},"
#     query_string += f"{response1_df['quotes'][0][i]['symbol']},"
query_string = query_string.rstrip(',')
query_string

'ADBE,QS,DIS,MRIN,FDX,UBER,RWLK,GTEK,DKNG,000001.SS'

# 2. REAL TIME QUOTE DATA

API domain: https://yfapi.net
API path: /v6/finance/quote
API variables:  querystring  {"symbols" : "string1,string2,string3,..."}

Pull data (extract) from yahoofinancials API (URL path is /v6/finance/quote) with "symbols" set to comma-separated list of up to 10 stock symbols at a time.

Use the query_string of 10 stock symbols.

In [28]:
## Query of /v6/finance/quote API

url = "https://yfapi.net/v6/finance/quote"
## Alternate url
# url = "https://rest.yahoofinanceapi.com/v6/finance/quote"

## Warning: no space between symbols
querystring = {"symbols" : query_string}

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [29]:
## Make API request (query string required for this API)
response2 = requests.request("GET", url, headers=headers, params=querystring).json()

# What does the response look like?  We will have to use indexing to get to the 'result' level
pprint.pprint(response2)

{'quoteResponse': {'error': None,
                   'result': [{'ask': 0.0,
                               'askSize': 13,
                               'averageAnalystRating': '1.7 - Buy',
                               'averageDailyVolume10Day': 2439616,
                               'averageDailyVolume3Month': 1575028,
                               'bid': 0.0,
                               'bidSize': 8,
                               'bookValue': 29.04,
                               'currency': 'USD',
                               'displayName': 'Adobe',
                               'earningsTimestamp': 1639652340,
                               'earningsTimestampEnd': 1639652340,
                               'earningsTimestampStart': 1639652340,
                               'epsCurrentYear': 12.24,
                               'epsForward': 14.04,
                               'epsTrailingTwelveMonths': 11.539,
                               'esgPopulated': False,
  

In [30]:
## How many records did we get?
len(response2['quoteResponse']['result'])

10

In [31]:
# Create a new DataFrame from all the results in the dictionary
response2_df = pd.DataFrame(response2['quoteResponse']['result'])
response2_df

,language,region,quoteType,quoteSourceName,triggerable,currency,firstTradeDateMilliseconds,priceHint,fiftyDayAverageChange,fiftyDayAverageChangePercent,...,priceEpsCurrentYear,sharesOutstanding,bookValue,fiftyDayAverage,marketCap,displayName,symbol,dividendDate,trailingAnnualDividendRate,trailingAnnualDividendYield
0,en-US,US,EQUITY,Delayed Quote,True,USD,524323800000,2,-1.426453,-0.002204,...,52.768790,4.764000e+08,29.040,647.316470,3.077020e+11,Adobe,ADBE,NaN,NaN,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,1597671000000,2,2.612061,0.121446,...,-50.250004,3.050400e+08,3.634,21.507940,1.000129e+10,QuantumScape,QS,NaN,NaN,NaN
2,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,-252322200000,2,-8.394119,-0.046747,...,69.865300,1.817130e+09,48.243,179.564120,3.110381e+11,NaN,DIS,1.579133e+09,0.88,0.004927
3,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,1363959000000,2,-0.110294,-0.019015,...,NaN,1.546670e+07,1.317,5.800294,8.800552e+07,NaN,MRIN,1.507248e+09,NaN,NaN
4,en-US,US,EQUITY,Delayed Quote,True,USD,261239400000,2,-15.520569,-0.058001,...,11.890094,2.661910e+08,90.399,267.590580,6.709877e+10,FedEx,FDX,1.633046e+09,2.70,0.010766
5,en-US,US,EQUITY,Delayed Quote,True,USD,1557495000000,2,3.392353,0.082806,...,-221.800000,1.884350e+09,7.539,40.967648,8.358977e+10,Uber,UBER,NaN,NaN,NaN
6,en-US,US,EQUITY,Delayed Quote,True,USD,1410528600000,4,1.226176,0.899073,...,-9.250000,4.641100e+07,1.430,1.363824,1.202045e+08,ReWalk Robotics,RWLK,1.554077e+09,NaN,NaN
7,en-US,US,ETF,Delayed Quote,True,USD,1631799000000,2,-0.363335,-0.009055,...,NaN,NaN,NaN,40.123333,NaN,NaN,GTEK,NaN,NaN,NaN
8,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,1564061400000,2,-3.742645,-0.066227,...,-18.515790,4.034220e+08,5.479,56.512646,2.128858e+10,DraftKings,DKNG,NaN,NaN,NaN
9,en-US,US,INDEX,Delayed Quote,False,CNY,867807000000,2,50.772460,0.014291,...,NaN,NaN,NaN,3552.635000,NaN,NaN,000001.SS,NaN,NaN,NaN


# 3. STOCK HISTORY

Open Source Library:  yahoo_fin
Usage: get_data(ticker, start_date = mm/dd/YYYY, end_date = None, index_as_date = True, interval = “1d”)
API variables:  ticker      a single ticker symbol or a list of ticker symbols
                start_date  first date in requested history, back to 1970
                end_date    last date in requested hostory, to the present
                index_as_date  should date column be used as the dataframe index?
                interval    data interval of 1d, 1w, 1m for daily, weekly, monthly

Pull data (extract) from yahoo_fin library with "ticker" set to a list of up to 10 stock symbols at a time.  Specify data interval and range of desired time span.

Returns one dataframe of data per ticker

In [32]:
## Query of yahoo_fin
from yahoo_fin.stock_info import get_data

my_tickers=stock_list
my_startdate="12/04/2009"
my_enddate="09/21/2021"
my_indexdate=True
my_interval="1wk"

In [33]:
## Make API request

response3 = {}
for ticker in my_tickers:
    response3[ticker] = get_data(ticker, 
                                 start_date=my_startdate,
                                 end_date=my_enddate,
                                 index_as_date=my_indexdate,
                                 interval=my_interval)

In [34]:
## How many records did we get?
len(response3)

20

In [35]:
# response3 is a dictionary of dataframes.  To access each, you should run a 'for'
## loop through it.  Here you can look at the 20 keys (stock symbols) for the latest
## request
response3.keys()

dict_keys(['ADBE', 'QS', 'DIS', 'MRIN', 'FDX', 'UBER', 'RWLK', 'GTEK', 'DKNG', '000001.SS', 'NNVC', 'SFIX', 'HLBZ', 'UPST', 'LCID', 'XELA', 'ABNB', 'UPS', 'AEMD', 'NURO'])

In [36]:
# Each member of the response3 dictionary is a dataframe
response3['DIS']

,open,high,low,close,adjclose,volume,ticker
2009-11-30,30.799999,31.100000,30.299999,30.840000,26.556301,23918100,DIS
2009-12-07,30.930000,31.830000,30.450001,31.700001,27.296848,59513000,DIS
2009-12-14,31.920000,32.590000,31.580000,31.950001,27.829397,66176900,DIS
2009-12-21,32.049999,32.500000,31.940001,32.290001,28.125551,26990200,DIS
2009-12-28,32.240002,32.750000,31.799999,32.250000,28.090706,40567900,DIS
...,...,...,...,...,...,...,...
2021-08-23,177.080002,180.539993,175.910004,180.139999,180.139999,31378100,DIS
2021-08-30,180.500000,185.300003,178.529999,181.000000,181.000000,35684100,DIS
2021-09-06,181.020004,187.580002,180.050003,184.119995,184.119995,32095100,DIS
2021-09-13,185.119995,185.899994,181.100006,183.470001,183.470001,34650800,DIS


# 4. Steps beyond

We could save the raw outputs into 3 database tables.

Transformations (either in the database or here in Jupyter Notebook):
The above new API pulls in dataframes with the proper timestamp format.  With this API we don't need to convert the time stamp

Transforming: renaming columns, taking fewer columns into new dataframe before loading, calculating new needed columns, dropping duplicates

Load (in the database):
Decide for each raw table above which columns will be needed for the final set of tables.  Make those tables, calculations, etc, and produce the final set of tables.  Set up connection string and method for how to transfer dataframe into database tables.

# 5. Connect to database
Use the values pulled in from config.py to construct the database connection string.  Connect to the database.

In [37]:
from sqlalchemy import create_engine

# Define the engine string to be used for connecting to your database using the 
#  environment variables found in config.py

database_url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
# database_url = f"postgresql://{username}:{password}@{host}:5432/{database}"
### Alternate URLs
# database_url = os.environ.get('DATABASE_URL', 'postgresql://localhost:5432/EmployeeSQL')
# connection_string = f'${username}:${password}@${host}:${port}/${database}'
# engine = create_engine(f'postgresql://{connection_string}')
engine = create_engine(database_url)
# connection = engine.connect()

In [38]:
# Example of how to work with subset of data into a new dataframe
# new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
# new_customer_data_df.head()

# Write dataframe to database, decide whether index is true or false, depending on
#   whether the index should be a column or not (possible primary key column)
# transformed_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)
# premise_transformed_df.to_sql(name='premise', con=engine, if_exists='append', index=True)

# Verify data presence by querying database
pd.read_sql_query('select * from departments', con=engine).head()

,dept_no,dept_name
0,d001,Marketing
1,d002,Finance
2,d003,Human Resources
3,d004,Production
4,d005,Development
